Advanced visualization using Tensorboard (weights, gradient, ...). This example is using the MNIST database of handwritten digits (http://yann.lecun.com/exdb/mnist/).




In [1]:
from __future__ import print_function, absolute_import

import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/tmp/data/", one_hot=True)


W0825 16:51:09.615310 139948700710784 deprecation.py:323] From <ipython-input-1-ec91bd7f9c48>:7: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0825 16:51:09.616667 139948700710784 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0825 16:51:09.617914 139948700710784 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for 

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz


W0825 16:51:10.012650 139948700710784 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0825 16:51:10.015499 139948700710784 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0825 16:51:10.112262 139948700710784 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: __init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a f

Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:
#training hyperparameters
learning_rate=0.01
training_epochs = 25
batch_size=100
display_step=1
logs_path='/tmp/tensorflow_logs/example/'

#network hyperparameters
n_hidden_1 = 256
n_hidden_2=256
n_input = 784
n_classes =10

#graph input
x=tf.placeholder(tf.float32, [None, 784], name='InputData')
y=tf.placeholder(tf.float32, [None, 10], name='LabelData')

In [0]:
#create model
def multilayer_perceptron(x, weights, biases):
  #hidden layer with relu
  layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
  layer_1 = tf.nn.relu(layer_1)
  
  #create summary to visualize first layer relu activation
  tf.summary.histogram("relu1", layer_1)
  
  #hidden layer with relu activation
  layer_2=tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
  layer_2=tf.nn.relu(layer_2)
  
  #create summary to visualize second layer relu activation
  tf.summary.histogram("relu2", layer_2)
  
  #output layer
  out_layer = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
  return out_layer

In [0]:
#store layer weights and biases
weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden_1]), name='W1'),
    'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name='W2'),
    'w3': tf.Variable(tf.random_normal([n_hidden_2, n_classes]), name='W3')    
}

biases={
    'b1': tf.Variable(tf.random_normal([n_hidden_1]), name='b1'),
    'b2': tf.Variable(tf.random_normal([n_hidden_2]), name='b2'),
    'b3': tf.Variable(tf.random_normal([n_classes]), name='b3')
}

In [5]:
# encapsulating all ops into scopes, making tensorboard's graph
# visualization more convenient
with tf.name_scope('Model'):
  #build model
  pred = multilayer_perceptron(x, weights, biases)
  
with tf.name_scope('Loss'):
  #softmax cross entropy function
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, 
                                                               labels=y))
  
with tf.name_scope('SGD'):
  # gradient descent
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  #op to calculate every variable gradient
  grads = tf.gradients(loss, tf.trainable_variables())
  grads = list(zip(grads, tf.trainable_variables()))
  #op to update all variables according to their gradients
  apply_grads = optimizer.apply_gradients(grads_and_vars=grads)
  
with tf.name_scope('Accuracy'):
  #accuracy
  acc = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
  acc = tf.reduce_mean(tf.cast(acc, tf.float32))

W0825 16:51:10.419634 139948700710784 deprecation.py:323] From <ipython-input-5-1f1d195e9bcd>:8: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [0]:
# initialize variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# create summary to monitor cost tensor
tf.summary.scalar("loss", loss)

#create summaries to monitor accuracy tensor
tf.summary.scalar("accuracy",acc)

#create summaries to visualize weights
for var in tf.trainable_variables():
  tf.summary.histogram(var.name, var)
  
#summarize all gradients
for grad, var in grads:
  tf.summary.histogram(var.name+'/gradient', grad)

# merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [8]:
# start training
with tf.Session() as sess:
  sess.run(init)
  
  # ops to write logs to tensorboard
  summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
  
  #training cycle
  for epoch in range(training_epochs):
    avg_cost=0.0
    total_batch = int(mnist.train.num_examples/batch_size)
    #loop over all batches
    for i in range(total_batch):
      batch_xs, batch_ys = mnist.train.next_batch(batch_size)
      # run optimization op, cost op and summary nodes
      _, c, summary = sess.run([apply_grads, loss, merged_summary_op],
                              feed_dict={x:batch_xs, y:batch_ys})
      #write every log iteration
      summary_writer.add_summary(summary, epoch*total_batch+i)
      #compute average loss
      avg_cost += c/total_batch
      
    #display logs per epoch step
    if (epoch+1)%display_step==0:
      print("epoch", epoch, "cost", avg_cost)
      
  print("Optimization finshed!")
  
  # test the model
  # calculate the accuracy
  print("accuracy:", acc.eval({x:mnist.test.images, y:mnist.test.labels}))      

epoch 0 cost 61.8510709116676
epoch 1 cost 13.593694000352503
epoch 2 cost 8.41824804686687
epoch 3 cost 5.970117229193297
epoch 4 cost 4.479069508083844
epoch 5 cost 3.47780684788972
epoch 6 cost 2.7843705368735447
epoch 7 cost 2.2987578085163287
epoch 8 cost 1.8782242421954858
epoch 9 cost 1.5734179393144336
epoch 10 cost 1.3306603835662096
epoch 11 cost 1.1554562907422754
epoch 12 cost 0.9506682364792144
epoch 13 cost 0.8413468652670374
epoch 14 cost 0.7048440762077913
epoch 15 cost 0.6172932620256838
epoch 16 cost 0.5368010000267492
epoch 17 cost 0.47011160531716717
epoch 18 cost 0.4030200679279594
epoch 19 cost 0.3641216773905867
epoch 20 cost 0.30790538458490174
epoch 21 cost 0.26706880995804144
epoch 22 cost 0.2447375206643178
epoch 23 cost 0.21580885219207666
epoch 24 cost 0.18589888564801987
Optimization finshed!
accuracy: 0.9295
